In [1]:
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = \
keras.datasets.fashion_mnist.load_data()

은닉층 - 신경망 성능 향상, 활성함수(2진분류-시그모이드) 등  

![image](https://user-images.githubusercontent.com/58851945/119298967-05e76100-bc99-11eb-9c4d-a2084768a2b4.png)

In [2]:
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0 
train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, train_target, val_target = \
train_test_split(train_scaled, train_target, test_size=.2, random_state=42)

In [3]:
# 은닉층 - 활성화함수 담을때, 모델 성능 높일때 사용 
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)) # 100개의 노드

# 출력층
dense2 = keras.layers.Dense(10, activation='softmax')  # hidden shape

# 심층신경망(DNN) 모델 - 은닉층을 가진 신경망
model = keras.Sequential([dense1, dense2])

은닉층(dense1)은 출력층(dense2) 보다 많아야한다
은닉층이 출력층보다 적으면 데이터가 손실될 수 있다
데이터 클래스가 10개라서 출력층에 10을 씀

층이 많아지면 name을 담기도 함
나중에 층을 쌓진않고 add라는 시퀀스 안에.... 뒤에 나옴
출력층은 제일 마지막에 와야 함

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               78500     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


`Output Shape` - `None`  
epoch랑 비슷  
배치사이즈 - 한번에 32개의 노드를 읽어와서 실행(디폴트)  
한번 반복할때마다 실행시키는 노드의 수
  
`Param`  
파라미터의 수  
78500개의 픽셀에 100개의 가중치를 줌 <= (784+1)x100 = (784개의 픽셀+절편1개)x100  
100x10 + 10  
　　　　절편

모델 이름(name)은 영문자로 쓰는걸 권장(한글x)

###### 층을 추가하는 방법들
model = keras.Sequential([  
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),  
    keras.layers.Dense(10, activation='softmax', name='output')  
    ], name='fashion MNIST model')  

model = keras.Sequential()  
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape(784,)))  
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()

In [5]:
# 모델 설정
model.compile(loss='sparse_categorical_crossentropy',  # 다중 분류
             metrics=['accuracy'])

# 모델 학습
model.fit(train_scaled, train_target, epochs=5)

Train on 48000 samples
Epoch 1/5
48000/48000 [==============================] - 3s 59us/sample - loss: 0.5599 - accuracy: 0.8099
Epoch 2/5
48000/48000 [==============================] - 2s 46us/sample - loss: 0.4067 - accuracy: 0.8537
Epoch 3/5
48000/48000 [==============================] - 2s 45us/sample - loss: 0.3732 - accuracy: 0.8650
Epoch 4/5
48000/48000 [==============================] - 2s 51us/sample - loss: 0.3514 - accuracy: 0.8721
Epoch 5/5
48000/48000 [==============================] - 2s 51us/sample - loss: 0.3344 - accuracy: 0.8777


In [6]:
(train_input, train_target), (test_input, test_target) = \
keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0 
# train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, train_target, val_target = \
train_test_split(train_scaled, train_target, test_size=.2, random_state=42)

In [7]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28))) 
# Flatten 유틸리티층. (평평하게하다, 단조롭게하다) -> 직접 1차원으로 만들어줌
# input_shape 입력층 다음에 적어야 함

model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)

Train on 48000 samples
Epoch 1/5
48000/48000 [==============================] - 3s 58us/sample - loss: 0.5239 - accuracy: 0.8146
Epoch 2/5
48000/48000 [==============================] - 3s 55us/sample - loss: 0.3895 - accuracy: 0.8601
Epoch 3/5
48000/48000 [==============================] - 3s 57us/sample - loss: 0.3569 - accuracy: 0.8711
Epoch 4/5
48000/48000 [==============================] - 3s 52us/sample - loss: 0.3363 - accuracy: 0.8795
Epoch 5/5
48000/48000 [==============================] - 2s 50us/sample - loss: 0.3221 - accuracy: 0.8861


In [10]:
model.evaluate(val_scaled, val_target, verbose=0)

[0.3666121663848559, 0.8746667]

SGD 확률적 경사하강법  Stochastic Gradient Descent
이동할 때 학습률(이동크기)를 조절 
적응적 학습률 : 거리가 먼 곳에선 step_size를 많이 주고, 가까운곳에선 학습률을 줄여서 step_size를 작게함  cf. 확률적 학습률
momentum 가속도(가속화). 디폴트 0

In [11]:
(train_input, train_target), (test_input, test_target) = \
keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0 
# train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, train_target, val_target = \
train_test_split(train_scaled, train_target, test_size=.2, random_state=42)

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [12]:
# 모델 설정 - adam
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)

Train on 48000 samples
Epoch 1/5
48000/48000 [==============================] - 2s 49us/sample - loss: 0.5261 - accuracy: 0.8170
Epoch 2/5
48000/48000 [==============================] - 2s 43us/sample - loss: 0.3963 - accuracy: 0.8573
Epoch 3/5
48000/48000 [==============================] - 2s 41us/sample - loss: 0.3549 - accuracy: 0.8715
Epoch 4/5
48000/48000 [==============================] - 2s 43us/sample - loss: 0.3307 - accuracy: 0.8790
Epoch 5/5
48000/48000 [==============================] - 2s 44us/sample - loss: 0.3116 - accuracy: 0.8858


In [13]:
model.evaluate(val_scaled, val_target, verbose=0)  # 검증된 정확도

[0.37170468040307364, 0.8631667]

In [14]:
(train_input, train_target), (test_input, test_target) = \
keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0 
# train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, train_target, val_target = \
train_test_split(train_scaled, train_target, test_size=.2, random_state=42)

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

# 모델 설정 - RMSprop
model.compile(optimizer='RMSprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)

Train on 48000 samples
Epoch 1/5
48000/48000 [==============================] - 3s 65us/sample - loss: 0.5322 - accuracy: 0.8119
Epoch 2/5
48000/48000 [==============================] - 3s 54us/sample - loss: 0.3939 - accuracy: 0.8581
Epoch 3/5
48000/48000 [==============================] - 3s 66us/sample - loss: 0.3569 - accuracy: 0.8727
Epoch 4/5
48000/48000 [==============================] - 3s 63us/sample - loss: 0.3336 - accuracy: 0.8804
Epoch 5/5
48000/48000 [==============================] - 3s 55us/sample - loss: 0.3196 - accuracy: 0.8860


In [15]:
(train_input, train_target), (test_input, test_target) = \
keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0 
# train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, train_target, val_target = \
train_test_split(train_scaled, train_target, test_size=.2, random_state=42)

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

# 모델 설정 - adagrad
model.compile(optimizer='adagrad', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)

Train on 48000 samples
Epoch 1/5
48000/48000 [==============================] - 3s 61us/sample - loss: 0.7790 - accuracy: 0.7541
Epoch 2/5
48000/48000 [==============================] - 3s 57us/sample - loss: 0.6075 - accuracy: 0.8020
Epoch 3/5
48000/48000 [==============================] - 2s 49us/sample - loss: 0.5667 - accuracy: 0.8160
Epoch 4/5
48000/48000 [==============================] - 2s 46us/sample - loss: 0.5438 - accuracy: 0.8224
Epoch 5/5
48000/48000 [==============================] - 2s 49us/sample - loss: 0.5280 - accuracy: 0.8271
